# Лабораторная работа 10
# Основы объектно-ориентированного программирования в Python. Проектирование классов

Шпак Андрей Валерьевич, 09.08.2022

# Задание 1

Проработайте материал лекции, представленный в файле $\textrm{2022_KM2_T10_OOP_design.html}$.

## Задание 1.1

Перенесите в свой блокнот Jupyter Notebook информационную структуру лекций, с указанием номера и темы раздела.

# Тема 10. Проектирование классов

## 10.1 Композиция

## 10.2 Делегирование

## 10.3 Абстрактный класс

## 10.4 Шаблон проектирования: синглтон

## 10.5 Классовые методы и статические методы

## 10.6 Особенности именования атрибутов и методов

## 10.7 Атрибут-свойство